# Dreambooth

## Build Environment

# Download the 1.5 model from hugging face
You can also provide your own v1.* model for training by uploading it and renaming it to "model.ckpt".  It should be in the same directory as dreambooth_runpod_joepenna.ipynb

In [ ]:
# Download the 1.5 sd model
from IPython.display import clear_output
from huggingface_hub import hf_hub_download

downloaded_model_path = hf_hub_download(
 repo_id="panopstor/EveryDream",
 filename="sd_v1-5_vae.ckpt"
)

# Move the sd_v1-5_vae.ckpt to the root of this directory as "model.ckpt"
actual_locations_of_model_blob = !readlink -f {downloaded_model_path}
!mv {actual_locations_of_model_blob[-1]} model.ckpt
clear_output()
print("✅ model.ckpt successfully downloaded")

# Regularization Images (Skip this section if you are uploading your own or using the provided images)

Training teaches your new model both your token **but** re-trains your class simultaneously.

From cursory testing, it does not seem like reg images affect the model too much. However, they do affect your class greatly, which will in turn affect your generations.

You can either generate your images here, or use the repos below to quickly download 1500 images.

In [ ]:
# GENERATE 200 images - Optional
self_generated_files_prompt = "person" #@param {type:"string"}
self_generated_files_count = 200 #@param {type:"integer"}

!python scripts/stable_txt2img.py \
 --seed 10 \
 --ddim_eta 0.0 \
 --n_samples 1 \
 --n_iter {self_generated_files_count} \
 --scale 10.0 \
 --ddim_steps 50 \
 --ckpt model.ckpt \
 --prompt {self_generated_files_prompt}

dataset=self_generated_files_prompt

!mkdir -p regularization_images/{dataset}
!mv outputs/txt2img-samples/*.png regularization_images/{dataset}

In [ ]:
# Zip up the files for downloading and reuse.
# Download this file locally so you can reuse during another training on this dataset
!apt-get install -y zip
!zip -r regularization_images.zip regularization_images/{dataset}

# Download pre-generated regularization images
We've created the following image sets

* `man_euler` - provided by Niko Pueringer (Corridor Digital) - euler @ 40 steps, CFG 7.5
* `man_unsplash` - pictures from various photographers
* `person_ddim` - provided by David Bielejeski - ddim @ 50 steps, CFG 10.0
* `woman_ddim` - provided by David Bielejeski - ddim @ 50 steps, CFG 10.0
* `artstyle` - provided by Hackmans - ddim @ 50 steps, CFG 10.0

`person_ddim` is recommended for training people


In [ ]:
#Download Regularization Images

dataset="person_ddim" #@param ["man_euler", "man_unsplash", "person_ddim", "woman_ddim", "artstyle"]
!git clone https://github.com/djbielejeski/Stable-Diffusion-Regularization-Images-{dataset}.git

!mkdir -p regularization_images/{dataset}
!mv -v Stable-Diffusion-Regularization-Images-{dataset}/{dataset}/*.* regularization_images/{dataset}

# Upload your training images
Upload 10-20 images of someone to

```
/workspace/Dreambooth-Stable-Diffusion/training_images
```

WARNING: Be sure to upload an *even* amount of images, otherwise the training inexplicably stops at 1500 steps.

*   2-3 full body
*   3-5 upper body
*   5-12 close-up on face

The images should be:

- as close as possible to the kind of images you're trying to make

## Training

If training a person or subject, keep an eye on your project's `logs/{folder}/images/train/samples_scaled_gs-00xxxx` generations.

If training a style, keep an eye on your project's `logs/{folder}/images/train/samples_gs-00xxxx` generations.

In [ ]:
# Training

# This isn't used for training, just to help you remember what your trained into the model.
project_name = "project_name"

# MAX STEPS
# How many steps do you want to train for?
max_training_steps = 2000

# Match class_word to the category of the regularization images you chose above.
class_word = "person" # typical uses are "man", "person", "woman"

# If you are training a person's face, set this to True
i_am_training_a_persons_face = False

flip_p_arg = 0.0 if i_am_training_a_persons_face else 0.5

# This is the unique token you are incorporating into the stable diffusion model.
token = "firstNameLastName"

# 0 Saves the checkpoint when max_training_steps is reached.
# 250 saves the checkpoint every 250 steps as well as when max_training_steps is reached.
save_every_x_steps = 0

reg_data_root = "/workspace/Dreambooth-Stable-Diffusion/regularization_images/" + dataset

!rm -rf training_images/.ipynb_checkpoints
!python "main.py" \
 --base "configs/stable-diffusion/v1-finetune_unfrozen.yaml" \
 -t \
 --actual_resume "model.ckpt" \
 --reg_data_root "{reg_data_root}" \
 -n "{project_name}" \
 --gpus 0, \
 --data_root "/workspace/Dreambooth-Stable-Diffusion/training_images" \
 --max_training_steps {max_training_steps} \
 --class_word "{class_word}" \
 --token "{token}" \
 --no-test \
 --flip_p {flip_p_arg} \
 --save_every_x_steps {save_every_x_steps}


## Copy and name the checkpoint file(s)

In [ ]:
import re

training_images = !find training_images/*
date_string = !date +"%Y-%m-%dT%H-%M-%S"

if save_every_x_steps == 0:
 # Copy the checkpoint into our `trained_models` folder
 directory_paths = !ls -d logs/*
 last_checkpoint_file = directory_paths[-1] + "/checkpoints/last.ckpt"
 file_name = date_string[-1] + "_" + \
             project_name + "_" + \
             str(len(training_images)) + "_training_images_" + \
             str(max_training_steps) + "_max_training_steps_" + \
             token + "_token_" + \
             class_word + "_class_word.ckpt"

 file_name = file_name.replace(" ", "_")

 !mkdir -p trained_models
 !mv {last_checkpoint_file} trained_models/{file_name}

 print("Download your trained model from trained_models/" + file_name + " and use in your favorite Stable Diffusion repo!")
else:
 directory_paths = !ls -d logs/*
 checkpoints_directory = directory_paths[-1] + "/checkpoints/trainstep_checkpoints"
 file_paths = !ls -d "{checkpoints_directory}"/*

 for i, original_file_name in enumerate(file_paths):
  # Remove the "epoch=000000-step=0000" text
  steps = re.sub(checkpoints_directory + "/epoch=\d{6}-step=0*", "", original_file_name)

  # Remove the .ckpt
  steps = steps.replace(".ckpt", "")

  # Setup the filename
  file_name = date_string[-1] + "_" + \
                  project_name + "_" + \
                  str(len(training_images)) + "_training_images_" + \
                  str(steps) + "_training_steps_" + \
                  token + "_token_" + \
                  class_word + "_class_word.ckpt"

  file_name = file_name.replace(" ", "_")

  # Make the directory and move the files into it.
  !mkdir -p trained_models
  !mv {original_file_name} trained_models/{file_name}

 print("Download your trained models from the 'trained_models' folder and use in your favorite Stable Diffusion repo!")

# Big Important Note!

The way to use your token is `<token> <class>` ie `joepenna person` and not just `joepenna`

## Generate Images With Your Trained Model!

In [ ]:
!python scripts/stable_txt2img.py \
 --ddim_eta 0.0 \
 --n_samples 1 \
 --n_iter 4 \
 --scale 7.0 \
 --ddim_steps 50 \
 --ckpt "/workspace/Dreambooth-Stable-Diffusion/trained_models/{file_name}" \
 --prompt "joepenna person as a masterpiece portrait painting by John Singer Sargent in the style of Rembrandt"